* Find file location on disk
  * Place under Git
  * Create new IHE-SDC Git repo for SDC Object Model Usage Samples
  * Create a folder for SDC XML files
  * Create "Assmblies" folder for imported assemblies
    * Copy in the latest SDC.Schema dll 
    * Copy in "C:\Program Files\Microsoft Visual Studio\2022\Enterprise\Common7\IDE\PublicAssemblies\Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll" //Microsoft.VisualStudio.TestTools.UnitTesting
    * Create Archive folder for junk notebooks

In [ ]:
//Import assemblies from files
#r "./Assemblies/SDC_CodeGeneratorTest.dll"  //SDC.Schema namespace
#r "./Assemblies/Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll"  //unit test framework, if we want to use Asserts etc.

using System;
using SDC.Schema;
//
    //using SDC;
    //using System.Runtime;
    //using System.IO;

    using Microsoft.VisualStudio.TestTools.UnitTesting;
    //using Newtonsoft.Json;
    //using System.Data;
    //using System.Diagnostics;
    //using System.Drawing;
    //using System.Linq;
    //using System.Xml; 

* Read in a large SDC file, and then deserialize as XML

In [ ]:
string path = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml";
var FD = SDC.Schema.FormDesignType.DeserializeFromXmlPath(path); //Deserialize
string myXML =  SdcSerializer<FormDesignType>.Serialize(FD); //Serialize
myXML

Create iterator to return nodes

In [ ]:
using System.Diagnostics;

////SDC Setup
BaseType.ResetSdcImport();
string path = @".\SDC XML\Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml";
var FD = SDC.Schema.FormDesignType.DeserializeFromXmlPath(path); //Deserialize

////TIMER SETUP
Stopwatch.StartNew();
var timerStartTime  = (float)Stopwatch.GetTimestamp();

int i = 0;

foreach (BaseType node in NodeIterator(FD))
{
    btPrint(node);
}
///////////////////////////////////////////////////
IEnumerable<BaseType> NodeIterator(BaseType? n)
    {
            while (n is not null)
            {
                n = MoveNext(n);
                if (n is not null)
                    yield return n;
                else yield break;
            }
        yield break;
    }
/////////////////////////////////////////////////
    BaseType? MoveNext(BaseType n)
    {
        Dictionary<Guid, List<BaseType>> cn = n.TopNode.ChildNodes;
        List<BaseType>? childList;
        BaseType? nextNode;
        
        n.order = i;  //almost instananeous
        Assert.IsTrue(n.ObjectID == i);//very fast
        i++;
        //if n has child nodes, the next node is the first child node of n.
        if (cn.TryGetValue(n.ObjectGUID, out childList))
        {
            nextNode = childList[0];
            if (nextNode is not null) return nextNode;
        }

        //n has no child nodes, so walk up the tree to the parent node.
        //When we walk back up the object graph, prevPar is the original starting node (deeper in the tree),
        //while par is the parent of prevPar, more superficial in the tree, and closer to the top node
        //We then check the childList of par, to see if prePar can be found in that childList
        //If prevPar is in childList, then try to retrieve the next node in childList
        //IF we cant get the nextNode from childList, then move up to one higher parent level
        var prevPar = n;  
        var par = prevPar.ParentNode;

        while (par is not null)
        {
            if (cn.TryGetValue(par.ObjectGUID, out childList))
            {
                var index = childList.IndexOf(prevPar);
                if (index < childList.Count - 1)
                {
                    nextNode = childList[index + 1];
                    if (nextNode is not null) return nextNode;
                }
                //the next node is not located yet, so walk up to a previous ancestor and try again,
                //looking in that ancestors childList for a nextNode candidate
                prevPar = par; 
                par = prevPar.ParentNode;
            }
        }
        return null;
    }

    void btPrint(BaseType n)
    {
        string content;
        if (n is not null)
        {
            if (n is DisplayedType) content = ": title: " + (n as DisplayedType)?.title;
            else if (n is PropertyType) content = ": " + (n as PropertyType)?.propName + ": " + (n as PropertyType)?.val;
            else content = "";

            display(n.ObjectID.ToString().PadLeft(4) + ": " + i.ToString().PadLeft(4) + ": " + (n.name ?? "").PadRight(20) + ": " + (n.ElementName ?? "").PadRight(25) + content);
        }
    }

display(((Stopwatch.GetTimestamp() - timerStartTime) / Stopwatch.Frequency).ToString());

## Find SDC node changes compared to the reference template
* Create 2 FormDesign OMs, "newOM" and "refOM"
* Iterate each node in newOM, and locate matcching node in refOM by ID, SGuid and/or name
  * For each newOM node, determine if it's 
    * new, 
    * has a different parent node, 
    * previous sib has changed.
  * Add the changed node's ID/name/sGUID and change type to a dictionary
* Compare each SDC property in newOM with the matching refOM node
* 